# Módulo 2 Actividad 5

*1. Para nuestros datos de supermercado, crear una lista de los clientes, con el gasto hecho en compras de cada uno. Si no ha comprado, debe aparecer con un 0*

In [ ]:
import pandas as pd

detalle = pd.read_csv('detalle.csv')
boletas = pd.read_csv('boletas.csv')
clientes = pd.read_csv('clientes.csv', encoding='latin1')
productos = pd.read_csv('productos.csv', encoding='latin1')

df_detalle = pd.DataFrame(detalle)
df_boletas = pd.DataFrame(boletas)
df_clientes = pd.DataFrame(clientes)
df_productos = pd.DataFrame(productos)



In [ ]:
df_detalle_con_precios = pd.merge(df_detalle, df_productos, left_on='codigo_producto', right_on='codigo', how='left')
df_detalle_con_precios.drop('codigo_producto', axis=1, inplace=True)
df_detalle_con_precios['Total'] = df_detalle_con_precios['cantidad'] * df_detalle_con_precios['precio']

total_boleta = df_detalle_con_precios.groupby('numero_boleta')['Total'].sum().reset_index()
total_boleta.rename(columns={'Total': 'total_boleta'}, inplace=True)
df_boletas_con_total = pd.merge(df_boletas, total_boleta, left_on='numero', right_on='numero_boleta', how='left')
df_boletas_con_total.drop('numero_boleta', axis=1, inplace=True)

total_por_cliente = df_boletas_con_total.groupby('rut_cliente')['total_boleta'].sum().reset_index()
total_por_cliente.rename(columns={'total_boleta': 'Total por Cliente'}, inplace=True)
df_clientes_con_total = pd.merge(df_clientes, total_por_cliente, left_on='rut', right_on='rut_cliente', how='left')
df_clientes_con_total.drop(['rut', 'direccion', 'comuna', 'fecha_nacimiento', 'telefono', 'email', 'rut_cliente'], axis=1, inplace=True)
df_clientes_con_total.fillna(0, inplace=True)

df_clientes_con_total


*2. Crea un DataFrame con datos de ventas por producto y región. Utiliza multi-índices para representar la información por año y trimestre. Luego, calcula la suma de ventas por trimestre para cada producto.*

In [82]:
df_productos.rename(columns={'nombre': 'producto'}, inplace=True)
df_clientes['comuna'].fillna('Sin Informacion', inplace=True)

df_boletas['fecha'] = pd.to_datetime(df_boletas['fecha'])

df_boletas_x_detalle = pd.merge(df_boletas, df_detalle, left_on='numero', right_on='numero_boleta')
df_boletas_x_detalle_x_producto = pd.merge(df_boletas_x_detalle, df_productos, left_on='codigo_producto', right_on='codigo')
df_todos = pd.merge(df_boletas_x_detalle_x_producto, df_clientes,left_on='rut_cliente', right_on='rut')

df_todos['anio'] = df_todos['fecha'].dt.year
df_todos['trimestre'] = df_todos['fecha'].dt.quarter

df_todos.drop(['numero', 'fecha', 'rut_cliente', 'numero_boleta', 'codigo_producto', 'codigo', 'precio', 'rut', 'nombre',
               'direccion', 'fecha_nacimiento', 'telefono', 'email'], axis=1, inplace=True)

ventas_por_producto = df_todos.groupby('producto')['cantidad'].sum().reset_index()
#ventas_por_producto.rename(columns={'cantidad': 'Total Ventas por Producto'}, inplace=True)

productoXregion = df_todos.pivot_table(index=['producto', 'comuna'], columns=['anio', 'trimestre'], values='cantidad',
                                          aggfunc='sum', fill_value=0)

df_productoXregion = pd.DataFrame(productoXregion)

#ventas_totales = ventas_por_producto.set_index('producto')['cantidad']
#df_productoXregion.loc[('Total Ventas', 'Total')] = ventas_totales

df_productoXregion



Empty DataFrame
Columns: []
Index: [(Aceite de cocina, Santiago, 6), (Aceite de cocina, Sin Informacion, 5), (Agua mineral, Iquique, 3), (Agua mineral, La Serena, 3), (Arroz, Santiago, 3), (Arroz, Santiago, 6), (Arroz, Sin Informacion, 6), (Arroz, Sin Informacion, 9), (Azúcar, Iquique, 7), (Azúcar, Puerto Montt, 3), (Azúcar, Santiago, 2), (Café, Antofagasta, 4), (Café, Sin Informacion, 1), (Café, Valparaíso, 8), (Carne de pollo, Sin Informacion, 6), (Fideos, Antofagasta, 5), (Fideos, Concepción, 5), (Fideos, Sin Informacion, 2), (Frutas frescas, Sin Informacion, 2), (Galletas, Antofagasta, 7), (Harina, Antofagasta, 1), (Harina, Concepción, 1), (Harina, Iquique, 3), (Harina, Santiago, 2), (Harina, Sin Informacion, 2), (Helado, Antofagasta, 4), (Helado, Temuco, 4), (Helado, Viña del Mar, 2), (Leche, Puerto Montt, 1), (Leche, Santiago, 1), (Mantequilla, Valparaíso, 4), (Pescado, Antofagasta, 4), (Pescado, Rancagua, 9), (Pescado, Valparaíso, 20), (Queso, Viña del Mar, 5), (Verduras, Iquique, 1), (Verduras, Rancagua, 5), (Verduras, Valparaíso, 10), (Yogur, Sin Informacion, 7), (Yogur, Sin Informacion, 8)]